# Creating agents

In [16]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
from dotenv import load_dotenv


load_dotenv(override=True) # load environment variables from .env file

True

In [17]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Writer Agent

In [18]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

## Task

In [19]:
task = '''
        Write a concise but engaging blogpost about GPUs.
        Make sure the blogpost is within 100 words.
       '''


In [20]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [21]:
print(reply)

**Unlocking the Power of GPUs: The Heartbeat of Modern Technology**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, AI, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences. With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, computation by computation.


## Simple review of the post

In [22]:
## reviewer agent
reviewer = AssistantAgent(
    name="Reviewer",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a reviewer. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [23]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):


        Write a concise but engaging blogpost about GPUs.
        Make sure the blogpost is within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Reviewer):

**Unlocking the Power of GPUs: The Heartbeat of Modern Technology**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, AI, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences. With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, computation by computation.

----------

## Adding multi-level reviews using nested chats

In [24]:
SEO_reviewer = AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="As an SEO expert, your role is to analyze and enhance content for optimal search engine performance. "
        "Focus on providing actionable recommendations that boost rankings and drive organic traffic. "
        "Limit your feedback to 3 key points, ensuring they are specific and directly applicable. "
        "Start each review by introducing yourself as an SEO Reviewer.",
)

In [25]:
grammatical_error_reviewer = AssistantAgent(
    name="Grammatical Error Reviewer",
    llm_config=llm_config,
    system_message="As a grammar specialist, your task is to meticulously examine content "
        "for grammatical errors, punctuation mistakes, and style inconsistencies. "
        "Provide up to 3 key points addressing the most significant grammatical issues. "
        "Ensure your feedback is specific, actionable, and includes examples where appropriate. "
        "Begin each review by introducing yourself as a Grammatical Error Reviewer.",
)

In [26]:
ethics_reviewer = AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="As an ethics specialist, your role is to evaluate content for ethical integrity "
        "and identify any potential moral concerns. "
        "Provide up to 3 specific, actionable recommendations to address ethical issues. "
        "Ensure your feedback is concise and directly applicable. "
        "Start each review by introducing yourself as an Ethics Reviewer.",
)

In [28]:
meta_reviewer = AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)